# German Traffic Sign Recognition

## GTSRB Dataset
The dataset features 43 different signs under various sizes, lighting conditions, occlusions and is very similar to real-life data. Training set includes about 39000 images while test set has around 12000 images. <br />
![GTSRB](https://chsasank.github.io/assets/images/traffic/classes.jpg)

In [ ]:
import os
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, losses
from keras.optimizers import Adam

## Data Augmentation

In [ ]:
train_dir = "/kaggle/input/gtsrb-german-traffic-sign/train"

In [ ]:
# ImageDataGenerator from keras.preprocessing
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    subset='training',
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    train_dir,
    subset='validation',
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

## Build Model

In [ ]:
input_shape = (30,30,3) # img_rows, img_colums, color_channels
num_classes = 43

In [ ]:
## Build Model
inputs = layers.Input(shape=input_shape)
# 1st Conv layer 
x = layers.Conv2D(32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(inputs)
x = layers.Conv2D(32, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# 2nd Conv layer        
x = layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# Fully Connected layer        
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
# Compile Model
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

## Train Model

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL =val_generator.n//val_generator.batch_size

In [ ]:
# Train Model
history = model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=10, validation_data=val_generator, validation_steps=STEP_SIZE_VAL) #, callbacks=[checkpoint])

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("epoch vs error")
plt.ylabel("Error")
plt.xlabel("Epoch")
plt.legend(["train","validation"])
plt.show()

## Evaluate Model

In [ ]:
score = model.evaluate_generator(val_generator, steps=STEP_SIZE_VAL)
print('VAL loss :', score[0])
print('VAL accuracy:', score[1])

## Test Model

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
df.head()

In [ ]:
from shutil import copyfile
src_dir  = "/kaggle/input/gtsrb-german-traffic-sign/"
test_dir = "./Dataset/Test/"
for i in range(len(df)):
    if not os.path.exists(test_dir+str(df.iloc[i].ClassId)):
        os.makedirs(test_dir+str(df.iloc[i].ClassId))
    copyfile(src_dir+df.iloc[i].Path, test_dir+str(df.iloc[i].ClassId)+'/'+df.iloc[i].Path[5:])

In [ ]:
!mkdir -p Dataset/Test

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(30,30),
    batch_size=32,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

In [ ]:
STEP_SIZE_TEST =test_generator.n//test_generator.batch_size

score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print('Test loss:', score[0])
print('Test accuracy:', score[1])